In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Flatten, Reshape
from keras.callbacks import EarlyStopping
from datetime import datetime
import mlflow
import mlflow.pyfunc
import yaml

In [6]:
from libs.ts_forecasting import load_and_preprocess_data
from libs.ts_forecasting import plot_loaded_data
from libs.ts_forecasting import build_tsmixer_model
from libs.ts_forecasting import train_tsmixer_model
from libs.ts_forecasting import generate_dataset
from libs.ts_forecasting import time_based_split
from libs.ts_forecasting import scale_data
from libs.ts_forecasting import inverse_transform_array
from libs.ts_forecasting import train_test_time_based_split
from libs.ts_forecasting import backtest_model
from libs.ts_forecasting import predict_test_set
from libs.ts_forecasting import forecast_data
from libs.ts_forecasting import mean_absolute_percentage_error
from libs.ts_forecasting import evaluate_test_set
from libs.ts_forecasting import plot_test_set_predictions_with_history
from libs.ts_forecasting import plot_future_forecast_with_history
from libs.ts_forecasting import log_experiment_to_mlflow
from libs.ts_forecasting import load_mlflow_experimental_model
from libs.ts_forecasting import load_mlflow_staged_model

In [2]:
# # Load Functions
# from libs.ts_forecasting import *

2023-10-07 17:32:01.792841: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-07 17:32:01.855869: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-07 17:32:01.855902: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-07 17:32:01.856162: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-07 17:32:01.900452: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-07 17:32:01.901325: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [ ]:
# Load general configurations
with open('config/general_config.yaml', 'r') as file:
    general_config = yaml.safe_load(file)

# Load project-specific configurations
with open('config/project_config.yaml', 'r') as file:
    project_config = yaml.safe_load(file)

In [ ]:
print("General Configurations Loaded:\n")
print(general_config)

In [ ]:
print("\nProject Level Configurations Loaded:\n")
print(project_config)

# Data Loading

In [ ]:
# List to contain the datasets we load
dataframe_list = []

# Inflation

https://es.tradingeconomics.com/argentina/inflation-cpi#calendar-table

In [10]:
data_1 = load_and_preprocess_data(general_config['csv_paths'][0], 
                                  general_config['features'][0], 
                                  project_config['years'], 
                                  project_config['points_per_year'])
dataframe_list.append(data_1)
plot_loaded_data(data_1, general_config['features'][0], "Value")

NameError: name 'general_config' is not defined

# LELIQs

https://www.estadisticasbcra.com/leliq#:~:text=%23%23%23%23%20%C3%9Altimo%0A%0A%23%23%23%2015084400%0A%0A2023

In [ ]:
data_2 = load_and_preprocess_data(general_config['csv_paths'][1], 
                                  general_config['features'][1], 
                                  project_config['years'], 
                                  project_config['points_per_year'])

# Special pre processing for this data
data_2[general_config['features'][1]] = (data_2[general_config['features'][1]] / 1e6).round(2)

dataframe_list.append(data_2)
plot_loaded_data(data_2, general_config['features'][1], "Value")

In [ ]:
# Combine data for model training
combined_dataframe = pd.concat(dataframe_list, axis=1)
data = combined_dataframe.copy()
data.columns = general_config['features']

# Keep a copy of the original data for later purposes
data_original = data.copy()

In [ ]:
data.head()

# Scaling

In [ ]:
# Scale the data
data, data_scalers = scale_data(data)

# Modelling

In [ ]:
# Train and Test Set Creation:
X_train, X_test, Y_train, Y_test, X, Y = train_test_time_based_split(np.array(data), 
                                                                     general_config['look_back'], 
                                                                     general_config['train_size'])

In [ ]:
# Create the TSMixer-inspired model
tsmixer_model = build_tsmixer_model(input_shape=(general_config['look_back'], 
                                                 len(general_config['features'])), 
                                    num_features=len(general_config['features']))
tsmixer_model.compile(optimizer='adam', loss='mean_squared_error')
tsmixer_model.summary()

In [ ]:
# Train the model
history = train_tsmixer_model(tsmixer_model, X_train, Y_train, X_test, Y_test, 
                              epochs=general_config['epochs'], 
                              batch_size=general_config['batch_size'], 
                              patience=general_config['patience'])

In [ ]:
# Make predictions over the test set
Y_test_original, predictions_original, predictions = predict_test_set(tsmixer_model, X_test, Y_test, 
                                                                      data_scalers, general_config['features'])

In [ ]:
# Plot the test set predictions with the history of the time series
plot_test_set_predictions_with_history(Y_train, Y_test, predictions, data_scalers, general_config['features'])

In [ ]:
# Metrics for the Test Set:
print("\n[Test Set] Evaluation:")
test_set_metrics = evaluate_test_set(Y_test, predictions, data_scalers, general_config['features'])

# Backtesting

In [ ]:
# Apply backtesting
predictions_bt, true_values_bt = backtest_model(tsmixer_model, X, Y, general_config['features'], data_scalers,
                                                plot_results=True, epochs=general_config['epochs'], 
                                                batch_size=general_config['batch_size'],
                                                patience=general_config['patience'])

In [ ]:
# Use the evaluate function for backtesting results
print("\n[Backtest] Evaluation:")
metrics_bt = evaluate_test_set(true_values_bt, predictions_bt, data_scalers, general_config['features'])

# Future Forecast (beyond Test Set)

In [ ]:
# Forecast with Production model:
forecasted_col_names = [f"forecasted_{feature}" for feature in general_config['features']]
forecasted_df = forecast_data(tsmixer_model, data, general_config['look_back'], 12, 
                              data_scalers, general_config['features'], columns=forecasted_col_names)

In [3]:
# Plot the forecast with history
history_and_forecast_df = pd.concat([data_original, forecasted_df], axis=0)
plot_future_forecast_with_history(history_and_forecast_df, general_config['features'])

# MLFlow Experiment Registration

In [ ]:
# MLFlow Experiment Registration and model saving

# MLFlow Configuration
metrics_to_log = {
    "Inflation - TestSet - MAE": test_set_metrics['inflacion_MAE'],
    "LELIQs - TestSet - MAE": test_set_metrics['leliqs_MAE'],
    "Inflation - TestSet - RMSE": test_set_metrics['inflacion_RMSE'],
    "LELIQs - TestSet - RMSE": test_set_metrics['leliqs_RMSE'],
    "Inflation - TestSet - MAPE": test_set_metrics['inflacion_MAPE'],
    "LELIQs - TestSet - MAPE": test_set_metrics['leliqs_MAPE'],
    "Inflation - Backtesting - MAE": metrics_bt['inflacion_MAE'],
    "LELIQs - Backtesting - MAE": metrics_bt['leliqs_MAE'],
    "Inflation - Backtesting - RMSE": metrics_bt['inflacion_RMSE'],
    "LELIQs - Backtesting - RMSE": metrics_bt['leliqs_RMSE'],
    "Inflation - Backtesting - MAPE": metrics_bt['inflacion_MAPE'],
    "LELIQs - Backtesting - MAPE": metrics_bt['leliqs_MAPE']
}

artifacts_to_log = ["plots/training_loss_plot.png", 
                    "plots/test_set_predictions.png", 
                    "plots/backtesting_predictions.png", 
                    "plots/future_forecast.png"]

if general_config['register_experiment']:
    log_experiment_to_mlflow(tsmixer_model, general_config['model_name'], 
                             general_config['experiment_version'], general_config['model_artifact_name'],
                             general_config['epochs'], general_config['batch_size'], 
                             metrics_to_log, artifacts_to_log, history_and_forecast_df)
    
    print("Experiment Registered!")

# Loading Favorite Experiment Model:

In [ ]:
# # Loading Favorite Experiment Model:
# favorite_run_id = '06663277c6ba4300bbfd5803e1cca34e'
# tsmixer_model = load_mlflow_experimental_model(favorite_run_id, general_config['model_artifact_name'])
# print(tsmixer_model)

# Registering Model

In [ ]:
if general_config['register_model']:
    mlflow.register_model(f"runs:/{favorite_run_id}/{general_config['model_artifact_name']}", 
                          f"{general_config['model_name']}_v{general_config['production_version']}")
    print("Model Registered!")

# Fetching the MLflow Model from the Model Registry

In [ ]:
# # Loading Staged Model:

# tsmixer_model = load_mlflow_staged_model(general_config['model_name'], 
#                                          general_config['production_version'], 
#                                          general_config['stage'])
# print(tsmixer_model)

# Check Model with Forecast again

In [ ]:
# Forecast with Production model:
forecasted_col_names = [f"forecasted_{feature}" for feature in general_config['features']]
forecasted_df = forecast_data(tsmixer_model, data, general_config['look_back'], 12, 
                              data_scalers, general_config['features'], columns= forecasted_col_names)

# Plot the forecast with history
history_and_forecast_df = pd.concat([data_original, forecasted_df], axis=0)
plot_future_forecast_with_history(history_and_forecast_df, general_config['features'])